In [1]:
from keras.datasets import reuters

Using TensorFlow backend.
C:\Users\rritesh\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
C:\Users\rritesh\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
C:\Users\rritesh\AppData\Local\Continuum\anaconda3\lib\site-packages\tensorflow\python\framework\dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
C:

In [2]:
(train_data, train_labels), (test_data, test_labels) = reuters.load_data(num_words=10000)

In [3]:
len(train_data)

8982

In [4]:
len(test_data)

2246

In [5]:
print(train_data[0])
print(train_data.shape)
type(train_data)

[1, 2, 2, 8, 43, 10, 447, 5, 25, 207, 270, 5, 3095, 111, 16, 369, 186, 90, 67, 7, 89, 5, 19, 102, 6, 19, 124, 15, 90, 67, 84, 22, 482, 26, 7, 48, 4, 49, 8, 864, 39, 209, 154, 6, 151, 6, 83, 11, 15, 22, 155, 11, 15, 7, 48, 9, 4579, 1005, 504, 6, 258, 6, 272, 11, 15, 22, 134, 44, 11, 15, 16, 8, 197, 1245, 90, 67, 52, 29, 209, 30, 32, 132, 6, 109, 15, 17, 12]
(8982,)


numpy.ndarray

In [6]:
word_index = reuters.get_word_index()
reverse_word_index = dict([(value, key) for (key, value) in word_index.items()])
decoded_newswire = ' '.join([reverse_word_index.get(i - 3, '?') for i in
train_data[0]])

In [7]:
train_labels[10]

3

In [8]:
'''Encoding the data'''
import numpy as np
def vectorize_sequences(sequences, dimension=10000):
    results = np.zeros((len(sequences), dimension))
    for i, sequence in enumerate(sequences):
        results[i, sequence] = 1.
    return results

x_train = vectorize_sequences(train_data)
x_test = vectorize_sequences(test_data)

'''one-hot encoding of the labels consists of 
embedding each label as an all-zero vector 
with a 1 in the place of the label index.'''

In [9]:
def to_one_hot(labels, dimension=46):
    results = np.zeros((len(labels), dimension))
    for i, label in enumerate(labels):
        results[i, label] = 1.
    return results

one_hot_train_labels = to_one_hot(train_labels)
one_hot_test_labels = to_one_hot(test_labels)

# OR 

# from keras.utils.np_utils import to_categorical
# one_hot_train_labels = to_categorical(train_labels)
# one_hot_test_labels = to_categorical(test_labels)

In [10]:
'''Build Network'''

from keras import models
from keras import layers
model = models.Sequential()
model.add(layers.Dense(64, activation='relu', input_shape=(10000,)))
model.add(layers.Dense(64, activation='relu'))
model.add(layers.Dense(46, activation='softmax'))

There are two other things you should note about this architecture:
1. You end the network with a Dense layer of size 46. This means for each input sample, the network will output a 46-dimensional vector. Each entry in this vector (each dimension) will encode a different output class.
2. The last layer uses a softmax activation. You saw this pattern in the MNIST example. It means the network will output a probability distribution over the 46 different output classes—for every input sample, the network will produce a 46-dimensional output vector, where output[i] is the probability that the sample belongs to class i. The 46 scores will sum to 1.

best loss function to use in this case is categorical_crossentropy.

In [11]:
model.compile(optimizer='rmsprop',
            loss='categorical_crossentropy',
            metrics=['accuracy'])

In [12]:
x_val = x_train[:1000]
partial_x_train = x_train[1000:]

y_val = one_hot_train_labels[:1000]
partial_y_train = one_hot_train_labels[1000:]

In [13]:
history = model.fit(partial_x_train,
                partial_y_train,
                epochs=20,
                batch_size=512,
                validation_data=(x_val, y_val))

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where

Train on 7982 samples, validate on 1000 samples
Epoch 1/20
7982/7982 [==============================] - 5s 631us/step - loss: 2.4111 - acc: 0.5355 - val_loss: 1.6123 - val_acc: 0.6470
Epoch 2/20
7982/7982 [==============================] - 4s 447us/step - loss: 1.3188 - acc: 0.7205 - val_loss: 1.2702 - val_acc: 0.7260
Epoch 3/20
7982/7982 [==============================] - 2s 199us/step - loss: 1.0017 - acc: 0.7854 - val_loss: 1.1427 - val_acc: 0.7510
Epoch 4/20
7982/7982 [==============================] - 2s 218us/step - loss: 0.7956 - acc: 0.8307 - val_loss: 1.0253 - val_acc: 0.7810
Epoch 5/20
7982/7982 [==============================] - 2s 214us/step - loss: 0.6366 - acc: 0.8608 - val_loss: 0.9779 - val_acc: 0.7960
Epoch 6/20
7982/7982 [==============================] - 2s 204us/step - loss: 0.5097 - acc: 0.8887 - val_loss: 0.9466 - val_acc: 0.8060
Epoch 7/20
7982/7982 [===================

In [14]:
results = model.evaluate(x_test, one_hot_test_labels)
results

2246/2246 [==============================] - 1s 338us/step


[1.1860936091398406, 0.7871772039180766]

In [16]:
type(x_test)

numpy.ndarray

In [17]:
'''Listing 3.22 Generating predictions for new data'''

predictions = model.predict(x_test)

# '''Each entry in predictions is a vector of length 46:'''
# print(predictions[0].shape)

# '''The coefficients in this vector sum to 1:'''
# print(np.sum(predictions[0]))

# '''The largest entry is the predicted class—the class with the highest probability:'''
# print(np.argmax(predictions[0]))

In [19]:
predictions.shape

(2246, 46)

In [21]:
np.argmax(predictions[1354])

16